In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy as sp
import h5py

import holodeck as holo
import holodeck.single_sources as ss
from holodeck.constants import YR, MSOL
from holodeck import utils, detstats, plot


import hasasia.sensitivity as hsen
import hasasia.sim as hsim

# Read in hdf file

In [ ]:
filepath = '/Users/emigardiner/GWs/holodeck/output/brc_output/ss56_redz_2023-05-30_uniform-07a_n1000_r500_f40'
filename = filepath+'/sam_lib.hdf5'
ssfile = h5py.File(filename, 'r')
print(list(ssfile.keys()))
fobs = ssfile['fobs'][:]
hc_ss_all = ssfile['hc_ss'][...]
hc_bg_all = ssfile['hc_bg'][...]
shape = hc_ss_all.shape
nsamp, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
print('N,F,R,L:', nsamp, nfreqs, nreals, nloudest)
ssfile.close()

# Find dur, cad, and dfobs

In [ ]:
dur = holo.librarian.DEF_PTA_DUR * YR # would be helpful to save this in hdf file
hifr = nfreqs/dur
cad = 1.0 / (2 * hifr)
fobs_cents = holo.utils.nyquist_freqs(dur, cad) 
fobs_edges = holo.utils.nyquist_freqs_edges(dur, cad)
dfobs = np.diff(fobs_edges)

assert np.all(np.isclose(fobs_cents, fobs)), "fobs arrays don't match!"
# print(fobs.shape, fobs_cents.shape)
# print(fobs, '\n', fobs_cents)
print(np.max(np.abs(fobs-fobs_cents)))


In [ ]:
print(nfreqs)

In [ ]:
print('samples (N):', nsamp, '\nreals (R):', nreals, '\nN,F,R,L:', hc_ss_all.shape)

# Find Best Sample

In [ ]:
# find best
# found this in a another notebook to be sample 30
# nn=30
fidx=1 # choose second bin
hc_ref15_10yr = 11.2*10**-15 
print(fobs[fidx]*YR)
nsort, fidx, hc_tt, hc_ref15 = detstats.rank_samples(hc_ss_all, hc_bg_all, fobs, fidx=1, 
                              hc_ref=hc_ref15_10yr, ret_all=True)
nn = nsort[0]
print(nn, fobs[fidx]*YR, hc_ref15)

# Calibrate PTA

In [ ]:
# modify these until dp_bg = .5
npsrs=50
sigma=7.85e-6

# build PTA
phis = np.random.uniform(0, 2*np.pi, size = npsrs)
thetas = np.random.uniform(np.pi/2, np.pi/2, size = npsrs)
# sigmas = np.ones_like(phis)*sigma
psrs = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=sigma,
                phi=phis, theta=thetas)

In [ ]:
dp_bg = detstats.detect_bg_pta(psrs, fobs, cad, hc_bg_all[nn])
print(np.mean(dp_bg), np.median(dp_bg), np.std(dp_bg))